In [1]:
import re
import os
import numpy as np
import scipy as sp
import mne
import nilearn.plotting as nplt
import nilearn.image as image
import pickle
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from sklearn.metrics import roc_auc_score
from nodestimation.node_estimate import Node
from nodestimation.timewindow import TimeWindow, sliding_window
import nibabel as nib
from nodestimation.parcellation import freesurf_dict
import matplotlib.pyplot as plt
import nodestimation.project.path as path
from nodestimation.project.actions import save, read, save_format

In [2]:
subjects_dir, subjects = path.found_subject_dir()
tree = path.build_resources_tree(subjects)
print(tree['B1C2'][1]['raw'])


# print(path.build_resources_tree(subjects))

Analysing project structure...
	Subject:  B1C2
		Trans file: ok
		Fwd file: ok
		Src file: ok
		Bem file: ok
		Raw file: ok
		Resec file: ok
	Files structure for B1C2 has been analysed. Files tree is built
./Source/Subjects/B1C2/B1C2_ii_run1_raw_tsss_mc_art_corr_raw.fif


In [3]:
from nodestimation.mneraw import read_original_raw, first_processing

conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
epochs_tmin, epochs_tmax = -15, 15
crop_time = 120
snr = 0.5  # use SNR smaller than 1 for raw data
lambda2 = 1.0 / snr ** 2
method = "sLORETA"
rfreq = 200
nfreq = 50
lfreq = 1
hfreq = 70

raw = read_original_raw('./', _subject_tree=tree['B1C2'])
print('original:')
print(raw.info)
fp_raw = first_processing(raw, lfreq, nfreq, hfreq, rfreq=rfreq, crop=crop_time, _subject_tree=tree['B1C2'])
print('processed:')
print(fp_raw.info)

        # if not 'raw' in subject_tree:
        #     raise OSError("No one of raw files are found. Raw file must have extension "
        #                   "\'.fif\' and contain \'raw\' in its name")
        # if not 'resec' in subject_tree and not 'mni-resec' in subject_tree:
        #     raise OSError("No one of resection files are found. Resection file must have extension "
        #                   "\'.nii\' or  \'.pkl\' and contain \'resec\' in its name")

Looking for original raw file in files tree...
Original raw file has been found. trying to read...
Opening raw data file ./Source/Subjects/B1C2/B1C2_ii_run1_raw_tsss_mc_art_corr_raw.fif...
    Range : 39000 ... 1257999 =     39.000 ...  1257.999 secs
Ready.
Successfully read
original:
<Info | 24 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 acq_stim: 7 800.000000 2000.000000
 bads: []
 ch_names: MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, MEG0132, ...
 chs: 204 GRAD, 102 MAG, 7 STIM, 60 EEG, 2 EOG, 1 ECG, 1 EMG, 18 MISC, 9 CHPI
 custom_ref_applied: False
 description: Vectorview system at moscow
 dev_head_t: MEG device -> head transform
 dig: 151 items (3 Cardinal, 4 HPI, 61 EEG, 83 Extra)
 events: 1 item (list)
 experimenter: meg
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 2 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2011-10-11 10:36:23 UTC

In [ ]:
root= './'

tree = os.walk(os.path.join(root, 'Source'))

raw_files = []
src_files = []
inv_files = []
bem_files = []
fwd_files = []
trans_files = []
epochs_files = []
ave_files = []
stc_files = []

subjects_found = False

for walk in tree:
    for file in walk[2]:
        if re.search(r'.*raw\.fif', file):
            raw_files.append(os.path.join(walk[0], file))
        if re.search(r'.*src.*\.fif', file) or re.search(r'.*source.*space.*\.fif', file):
            src_files.append(os.path.join(walk[0], file))
        if re.search(r'.*inv.*\.fif', file) or re.search(r'.*inverse.*\.fif', file):
            inv_files.append(os.path.join(walk[0], file))
        if re.search(r'.*bem.*\.fif', file):
            bem_files.append(os.path.join(walk[0], file))
        if re.search(r'.*fwd.*\.fif', file) or re.search(r'.*forward.*\.fif', file):
            fwd_files.append(os.path.join(walk[0], file))
        if re.search(r'.*trans.*\.fif', file):
            trans_files.append(os.path.join(walk[0], file))
        if re.search(r'.*epo.*\.fif', file):
            epochs_files.append(os.path.join(walk[0], file))
        if re.search(r'.*ave.*\.fif', file):
            ave_files.append(os.path.join(walk[0], file))
        if re.search(r'.*stc.*\.fif.*', file):
            stc_files.append(os.path.join(walk[0], file))

    for subdir in walk[1]:
        if subdir == 'Subjects' or subdir == 'subjects' and not subjects_found:
            subjects_found = True
            subjects_dir = os.path.join(walk[0], subdir)
        elif subdir == 'Subjects' or subdir == 'subjects' and subjects_found:
            raise OSError("There are two subjects directories: {}, {}; Only one must be".format(
                subjects_dir, os.path.join(walk[0], subdir)
            ))

if not raw_files:
    raise OSError("No one of raw files are found. Raw file must have extension \'.fif\' and ends with \'raw\'")

if not subjects_found:
    raise OSError("Subjects directory not found!")
subjects = os.listdir(subjects_dir)

subject_dirs = []

for subject in subjects:
    subject_dirs.append(os.path.join(subjects_dir, subject))

raw_file = raw_files[0]
bem_file = bem_files[0]
src_file = src_files[0]
fwd_file = fwd_files[0]
trans_file = trans_files[0]

res_folder = os.path.join(root, 'Pipeline', subjects[0])

res_raw_folder = os.path.join(res_folder, 'Raw')
res_bem_folder = os.path.join(res_folder, 'Bem')
res_src_folder = os.path.join(res_folder, 'Src')
res_fwd_folder = os.path.join(res_folder, 'Fwd')
res_events_folder = os.path.join(res_folder, 'Events')
res_epochs_folder = os.path.join(res_folder, 'Epochs')
res_evoked_folder = os.path.join(res_folder, 'Evoked')
res_cov_folder = os.path.join(res_folder, 'Cov')
res_inv_folder = os.path.join(res_folder, 'Inv')
res_sLORETA_folder = os.path.join(res_folder, 'sLORETA')
res_nodes_folder = os.path.join(res_folder, 'NodesEstimate')
res_resec_folder = os.path.join(res_folder, 'Resection')

res_raw_file = os.path.join(res_raw_folder, 'raw.fif')
res_bem_file = os.path.join(res_bem_folder, 'raw_bem.fif')
res_src_file = os.path.join(res_src_folder, 'raw_src_ico5.fif')
res_fwd_file = os.path.join(res_fwd_folder, 'raw_fwd_ico5.fif')
res_events_file = os.path.join(res_events_folder, 'raw_eve.fif')
res_epochs_file = os.path.join(res_epochs_folder, 'raw_epo.fif')
res_evoked_file = os.path.join(res_evoked_folder, 'raw_ave.fif')
res_cov_file = os.path.join(res_cov_folder, 'noise_cov.fif')
res_inv_file = os.path.join(res_inv_folder, 'raw_inv.fif')
res_sLORETA_file = os.path.join(res_sLORETA_folder, 'sLORETA_raw_ave_inv.pkl')
res_nodes_strength_file = os.path.join(res_nodes_folder, 'nodes_strength_auc.dat')
res_pearson_nodes_file = os.path.join(res_nodes_folder, 'pearson_nodes.pkl')
res_plv_nodes_file = os.path.join(res_nodes_folder, 'plv_nodes.pkl')
res_resec_file = os.path.join(res_resec_folder, 'resection.pkl')

subject_dir = subject_dirs[0]
subject = subjects[0]

In [ ]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
epochs_tmin, epochs_tmax = -15, 15
crop_time = 120
snr = 0.5  # use SNR smaller than 1 for raw data
lambda2 = 1.0 / snr ** 2
method = "sLORETA"
rfreq = 200
nfreq = 50
lfreq = 1
hfreq = 70

In [ ]:
from os import mkdir


In [ ]:
if not os.path.exists('./Pipeline'):
    mkdir('./Pipeline')

if not os.path.exists(res_folder):
    mkdir(res_folder)

In [ ]:
if os.path.isfile(res_raw_file):
    raw = mne.io.read_raw_fif(res_raw_file)

elif os.path.isfile(raw_file):
    raw = mne.io.read_raw_fif(raw_file)
    raw.resample(rfreq, npad='auto')
    raw = first_processing(raw)
    path = res_raw_folder

    mkdir(path)

    raw.save(res_raw_file)

else:
    raise OSError('PIPELINE: Raw-file not found')


print(raw.info)

In [ ]:
if os.path.isfile(res_src_file):
    src = mne.read_source_spaces(res_src_file)

elif os.path.isfile(src_file):
    src = mne.read_source_spaces(src_file)
    path = res_src_folder

    mkdir(path)

    src.save(res_src_file)

else:
    print('PIPELINE: Source spaces not found, creating a new one...')
    src = mne.setup_source_space(subject, spacing='ico5', add_dist='patch', subjects_dir=subjects_dir)
    path = res_src_folder

    mkdir(path)

    src.save(res_src_file)

In [ ]:
if os.path.isfile(res_bem_file):
    bem = mne.read_bem_solution(res_bem_file)

elif os.path.isfile(bem_file):
    bem = mne.read_bem_solution(bem_file)
    path = res_bem_folder

    mkdir(path)

    mne.write_bem_solution(res_bem_file, bem)

else:
    print('PIPELINE: BEM-surface not found, creating a new one...')
    model = mne.make_bem_model(subject=subject, ico=4, conductivity=conductivity, subjects_dir=subject_dir)
    bem = mne.make_bem_solution(model)
    path = res_bem_folder

    mkdir(path)

    mne.write_bem_solution(res_bem_file, bem)

In [ ]:
if os.path.isfile(res_fwd_file):
    fwd = mne.read_forward_solution(res_fwd_file)

elif os.path.isfile(fwd_file):
    fwd = mne.read_forward_solution(fwd_file)
    path = res_fwd_folder

    mkdir(path)

    mne.write_forward_solution(res_fwd_file, fwd)

else:
    print('PIPELINE: Forward solution not found, creating a new one...')
    fwd = mne.make_forward_solution(res_raw_file, trans=trans_file, src=src, bem=bem, meg=True, eeg=False,
                                    mindist=5.0, n_jobs=1, verbose=True)
    path = res_fwd_folder

    mkdir(path)

    mne.write_forward_solution(res_fwd_file, fwd)

In [ ]:
events = np.array([[
        raw.first_samp + raw.time_as_index(i)[0],
        0,
        1
    ] for i in range(1, 59)])

In [ ]:
if os.path.isfile(res_epochs_file):
    epochs = mne.read_epochs(res_epochs_file)

else:
    print('PIPELINE: Epochs not found, creating a new one...')
    epochs = mne.Epochs(raw, events, tmin=-1, tmax=1,
                        preload=True)
    path = res_epochs_folder

    mkdir(path)

    epochs.save(res_epochs_file)

In [ ]:
if os.path.isfile(res_evoked_file):
    evoked = mne.read_evokeds(res_evoked_file)
else:
    print('PIPELINE: Evokeds not found, creating a new one...')
    evoked = epochs.average()
    path = res_evoked_folder

    mkdir(path)

    mne.write_evokeds(res_evoked_file, evoked)

    evoked = [evoked]

In [ ]:
# evoked[0].plot()

In [ ]:
if os.path.isfile(res_cov_file):
    noise_cov = mne.read_cov(res_cov_file)

else:
    print('PIPELINE: Noise covariance not found, creating a new one...')
    noise_cov = mne.compute_covariance(epochs.copy().pick_types(meg=True, eeg=False, eog=False), tmin=-1, tmax=0,
                                       method='empirical')
    path = res_cov_folder

    mkdir(path)

    mne.write_cov(res_cov_file, noise_cov)

In [ ]:
if os.path.isfile(res_inv_file):
    inv = mne.minimum_norm.read_inverse_operator(res_inv_file)

else:
    print('PIPELINE: Inverse operator not found, creating a new one...')
    inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, noise_cov, depth=None, fixed=False)
    path = res_inv_folder

    mkdir(path)

    mne.minimum_norm.write_inverse_operator(res_inv_file, inv)

In [ ]:


if os.path.isfile(res_sLORETA_file):
    print('Reading sLORETA solution...')
    stc = pickle.load(open(res_sLORETA_file, 'rb'))
    print('sLORETA has been read')

else:
    print('PIPELINE: sLORETA not found, creating a new one...')
    stc = mne.minimum_norm.apply_inverse_epochs(epochs,
                                         inv,
                                         lambda2,
                                         'sLORETA',
                                         pick_ori=None
                                         )
    path = res_sLORETA_folder

    mkdir(path)
    pickle.dump(stc, open(res_sLORETA_file, 'wb'))

In [ ]:
labels_parc = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)

# label_tc = stc.extract_label_time_course(labels_parc, src, mode='mean_flip')

label_ts = mne.extract_label_time_course(stc, labels_parc, src, mode='mean_flip')

In [ ]:
print(label_ts[0].shape)

plt.plot(np.arange(401), label_ts[0][10].T, np.arange(401), label_ts[0][36].T)
plt.show()
plt.plot(np.arange(401), label_ts[1][10].T, np.arange(401), label_ts[1][36].T)
plt.show()
plt.plot(np.arange(401), label_ts[2][10].T, np.arange(401), label_ts[2][36].T)
plt.show()
plt.plot(np.arange(401), label_ts[3][10].T, np.arange(401), label_ts[3][36].T)
plt.show()

In [ ]:
labels_aseg = mne.get_volume_labels_from_src(src, subject, subjects_dir)

labels = labels_parc + labels_aseg

label_names = [label.name for label in labels]

lh_labels = [name for name in label_names if name.endswith('lh')]

rh_labels = [name for name in label_names if name.endswith('rh')]

In [ ]:
fmin = 30
fmax = 70
sfreq = 200

con, freqs, times, n_epochs, n_tapers = mne.connectivity.spectral_connectivity(
    label_ts, method='plv', mode='multitaper', sfreq=sfreq, fmin=fmin,
    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

In [ ]:
label_ypos_lh = list()

for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos_lh.append(ypos)

try:
    idx = label_names.index('Brain-Stem')

except ValueError:
    pass

else:
    ypos = np.mean(labels[idx].pos[:, 1])
    lh_labels.append('Brain-Stem')
    label_ypos_lh.append(ypos)


lh_labels = [label for (yp, label) in sorted(zip(label_ypos_lh, lh_labels))]

rh_labels = [label[:-2] + 'rh' for label in lh_labels
             if label != 'Brain-Stem' and label[:-2] + 'rh' in rh_labels]


node_colors = [label.color for label in labels]

node_order = lh_labels[::-1] + rh_labels

node_angles = mne.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) // 2])
conmat = con[:, :, 0]

In [ ]:
for i in range(67):
    print((conmat[:, i]))

In [ ]:
fig = plt.figure(num=None, figsize=(8, 8), facecolor='black')
mne.viz.plot_connectivity_circle(conmat, label_names, n_lines=300,
                         node_angles=node_angles, node_colors=node_colors,
                         title='All-to-All Connectivity Epilepsy '
                         'Condition (PLV)', fig=fig)

In [ ]:
# fmin = 30.
# fmax = 50.
# sfreq = 200
#
# con, freqs, times, n_epochs, n_tapers = mne.connectivity.spectral_connectivity(
#     label_ts, method='plv', mode='multitaper', sfreq=sfreq, fmin=fmin,
#     fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)
#

In [ ]:
# fig = plt.figure(num=None, figsize=(8, 8), facecolor='black')
# mne.viz.plot_connectivity_circle(conmat, label_names, n_lines=300,
#                          node_angles=node_angles, node_colors=node_colors,
#                          title='All-to-All Connectivity Epilepsy '
#                          'Condition (PLV)', fig=fig)

In [ ]:
# nodes strength

# plt.plot(n_strength, 'o')
# plt.title('Node Strength')
# plt.xlabel('node: number')
# plt.ylabel('node: strength')
# plt.show()

In [ ]:
## compute roc curve
# resected_nodes = 15
#
# label_ind = np.zeros(len(n_strength))
# label_ind[0:resected_nodes] = True
# label_ind[resected_nodes+1:] = False
# Drs = roc_auc_score(label_ind, n_strength)

In [ ]:
# if os.path.isfile(res_pearson_nodes_file):
#     print('Reading nodes...')
#     nodes = pickle.load(open(res_pearson_nodes_file, 'rb'))
#
# else:
#     print('PIPELINE: Pearson\'s Nodes file not found, create a new one')
#
#     if not os.path.exists(res_nodes_folder):
#         mkdir(res_nodes_folder)
#
#     nodes = []
#     n_strength, pearson_connectome = nodes_strength(label_ts, 'pearson')
#
#     for i in range(len(n_strength)):
#         nodes.append(Node(label_ts[i, :], n_strength[i], labels[i], 'Pearson', pearson_connectome[i, :]))
#
#     pickle.dump(nodes, open(res_pearson_nodes_file, 'wb'))
#
# coordinates = []
# n_strength = []
# for node in nodes:
#     coordinates.append(node.nilearn_coordinates)
#     n_strength.append(node.strength)
#
# nplt.plot_markers(n_strength, coordinates, node_cmap='black_red_r')
# nplt.show()
#

In [ ]:
# if os.path.isfile(res_plv_nodes_file):
#     print('Reading nodes...')
#     nodes = pickle.load(open(res_plv_nodes_file, 'rb'))
#
# else:
#     print('PIPELINE: PLV Nodes file not found, create a new one')
#
#     if not os.path.exists(res_nodes_folder):
#         mkdir(res_nodes_folder)
#
#     nodes = []
#     n_strength, plv_connectome = nodes_strength(label_ts, 'plv')
#
#     for i in range(len(n_strength)):
#         nodes.append(Node(label_ts[i, :], n_strength[i], labels[i], 'PLV', plv_connectome[i, :, :]))
#
#     pickle.dump(nodes, open(res_plv_nodes_file, 'wb'))
#
# coordinates = []
# n_strength = []
# for node in nodes:
#     coordinates.append(node.nilearn_coordinates)
#     n_strength.append(node.strength)
#
# nplt.plot_markers(n_strength, coordinates, node_cmap='black_red_r')
# nplt.show()

In [ ]:
## example how to get freesurf_dict

# vertexes = [mne.vertex_to_mni(
#     label.vertices,
#     hemis=0 if label.hemi == 'lh' else 1,
#     subject=subject, subjects_dir=subjects_dir
# )for label in labels]
# freesurf_dict_sample = {l[0].name: np.mean(l[1], axis=0) for l in zip(labels, vertexes)}

In [ ]:
# # show one label
# nplt.plot_markers(np.zeros(vertexes[0].shape[0]), vertexes[0])
# nplt.show()

In [ ]:
# # show one node
#
# nplt.plot_markers(np.array([0, 0]), np.array([
#     np.mean(vertexes[0], axis=0),
#     np.array([1000, 1000, 1000]) ## plot markers does not work with one node
# ]))
# nplt.show()

In [ ]:
vertexes = [mne.vertex_to_mni(
    label.vertices,
    hemis=0 if label.hemi == 'lh' else 1,
    subject=subject, subjects_dir=subjects_dir
)for label in labels]

freesurf_dict_sample = {l[0].name: np.mean(l[1], axis=0) for l in zip(labels, vertexes)}
node_coordinates = np.array(list(freesurf_dict_sample.values()))

# print(node_coordinates)

In [ ]:
if os.path.isfile(res_resec_file):
    print('Reading resection coordinates...')
    resec_coordinates = pickle.load(open(res_resec_file, 'rb'))
    print('Resection coordinates has been read')
else:
    print('PIPELINE: Resection file not found, create a new one')
    if not os.path.exists(res_resec_folder):
        mkdir(res_resec_folder)

    img = nib.load('Source/Subjects/B1C2/resection/resection.nii')
    res = np.array(img.get_data().tolist())
    img_coordinates = list()
    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            for k in range(res.shape[2]):
                if res[i,j,k] != 0:
                    img_coordinates.append(np.array([i, j, k]))
    img_coordinates = np.array(img_coordinates)
    mni_coordinates = []
    for coordinate in img_coordinates:
        mni_coordinates.append(
            np.array(
                image.coord_transform(
                    coordinate[0],
                    coordinate[1],
                    coordinate[2],
                    img.affine
                    )
            )
        )
    
    resec_coordinates = np.array(mni_coordinates)
    pickle.dump(resec_coordinates, open(res_resec_file, 'wb'))

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

display = nplt.plot_glass_brain(None, display_mode='lyrz', figure=fig, axes=ax)

display.add_markers(resec_coordinates, marker_color="violet", marker_size=1)

display.add_markers(node_coordinates, marker_color="yellow", marker_size=30)

In [ ]:
spared = list()
resected = list()

for node_coordinate in node_coordinates:
    for resec_coordinate in resec_coordinates:
        diff = node_coordinate - resec_coordinate
        dist = np.sqrt(diff[0]**2 + diff[1]**2 + diff[2]**2)
        if dist <= 1 and not node_coordinate in np.array(resected):
            resected.append(node_coordinate)
        else:
            spared.append(node_coordinate)

fig, ax = plt.subplots(figsize=(15,15))


display = nplt.plot_glass_brain(
    None, display_mode='lyrz', figure=fig, axes=ax)
display.add_markers(resec_coordinates, marker_color="violet", marker_size=1)
display.add_markers(np.array(spared), marker_color="yellow", marker_size=100)
display.add_markers(np.array(resected), marker_color="red", marker_size=250)

In [ ]:
del stc, src, raw, fwd,\
    bem, labels, label_ts, inv, noise_cov
